In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# word embedding

In [15]:
# 关于word embedding， 以序列建模为例
# 考虑source sentence 和target sentence
# 构建序列，序列字符以索引形式表示
batch_size = 2

# 单词表大小
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8#源模型是512，embedding维度
# 最大序列长度
max_src_seq_len = 5
max_tgt_seq_len = 5
# src_len = torch.randint(2,5,(batch_size,))
# tat_len = torch.randint(2,5,(batch_size,))
src_len = torch.Tensor([2,4]).to(torch.int32)
tgt_len = torch.Tensor([4,3]).to(torch.int32)

# 单词索引构成的句子
src_seq = [torch.randint(1,max_num_src_words,(L,)) for L in src_len]# 单词索引序列
tgt_seq = [torch.randint(1,max_num_src_words,(L,)) for L in tgt_len]
print(src_seq,tgt_seq)
# 构建batch,单词索引构成源句子和目标句子，并做了padding，默认值为0
src_seq = [F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L))for L in src_len]# 元组(0,max_src_seq_len-L)：左边pad多少位右边pad多少位
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)),0) for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_tgt_words,(L,)),(0,max_tgt_seq_len-L)),0) for L in tgt_len])
print(src_seq,tgt_seq)

# 构造embedding
src_embedding_table = nn.Embedding(max_num_src_words+1,model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1,model_dim)

print(src_embedding_table.weight)

src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)
tgt_embedding

[tensor([2, 6]), tensor([5, 7, 2, 6])] [tensor([5, 4, 6, 4]), tensor([4, 6, 2])]
tensor([[4, 5, 0, 0, 0],
        [4, 7, 4, 1, 0]]) tensor([[1, 6, 5, 4, 0],
        [7, 5, 7, 0, 0]])
Parameter containing:
tensor([[ 0.5007, -0.6072,  1.4729,  0.7890, -2.5997,  0.8715, -1.5055,  0.4766],
        [-1.6439, -0.0193, -2.2346,  0.2380,  0.1550,  0.7215,  0.9870,  0.2234],
        [-0.3531, -0.1405,  2.8583, -0.5569,  0.8372,  0.8181, -2.1133, -1.4057],
        [ 1.0030, -0.7196,  0.5070, -1.4659,  1.1403, -0.5032, -0.1697,  0.7919],
        [-0.2741,  0.8439, -0.1827,  0.7454,  0.6124,  1.8913, -1.1633, -1.0219],
        [-1.0601, -0.5914, -0.0971, -0.0617,  0.1567,  0.4215,  0.1492, -0.7813],
        [ 1.3599, -0.0714,  0.7039,  0.1371,  1.2803,  0.4825, -1.3490, -0.2893],
        [ 1.0081,  0.2055,  0.1695,  1.7350, -0.4631,  0.6384,  0.4213, -0.8453],
        [-0.0170, -0.0956,  0.0819, -0.7202,  0.3002, -0.9604, -0.6468,  0.4037]],
       requires_grad=True)


tensor([[[-1.5677, -0.3175,  0.0145,  0.8478,  0.5012,  0.3512, -0.5572,
           0.4904],
         [ 0.1843, -0.5942,  1.9065,  0.1691,  1.1364, -0.5440, -0.1600,
           0.1765],
         [-2.4392, -0.4121,  0.1469,  1.6484, -0.0195, -0.9276,  0.4236,
           0.9293],
         [-1.5272,  0.4483, -0.4281,  0.6895,  1.9367,  1.8631, -0.7039,
           0.2041],
         [-0.0370, -0.7241,  0.0857,  0.2976,  0.3212, -0.5430, -0.6288,
          -0.0308]],

        [[ 1.5621,  0.6090, -0.6689, -1.0925,  0.2479, -0.2435, -1.8772,
           0.7904],
         [-2.4392, -0.4121,  0.1469,  1.6484, -0.0195, -0.9276,  0.4236,
           0.9293],
         [ 1.5621,  0.6090, -0.6689, -1.0925,  0.2479, -0.2435, -1.8772,
           0.7904],
         [-0.0370, -0.7241,  0.0857,  0.2976,  0.3212, -0.5430, -0.6288,
          -0.0308],
         [-0.0370, -0.7241,  0.0857,  0.2976,  0.3212, -0.5430, -0.6288,
          -0.0308]]], grad_fn=<EmbeddingBackward>)

In [28]:
# 构造position embedding
max_position_len = 6
pos_mat = torch.arange(max_position_len).reshape((-1,1))
print(pos_mat)
i_mat = torch.pow(10000,torch.arange(0,8,2).reshape((1,-1))/model_dim)
print(i_mat)
pe_embedding_table = torch.zeros(max_position_len,model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat/i_mat)#偶数列
pe_embedding_table[:, 1::2] = torch.cos(pos_mat/i_mat)#奇数列
print(pe_embedding_table)
pe_embedding = nn.Embedding(max_position_len,model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table,requires_grad=False)
print(pe_embedding.weight)
src_pos = torch.cat([torch.unsqueeze(torch.arange(max_src_seq_len),0) for _ in src_len]).to(torch.int32)
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max_tgt_seq_len),0) for _ in tgt_len]).to(torch.int32)

src_pe_embedding = pe_embedding(src_pos)
tgt_pe_emebdding = pe_embedding(tgt_pos)
print(tgt_pe_emebdding)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]])
tensor([[   1.,   10.,  100., 1000.]])
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01],
        [-9.5892e-01,  2.8366e-01,  4.7943e-01,  8.7758e-01,  4.9979e-02,
          9.9875e-01,  5.0000e-03,  9.9999e-01]])
Parameter containing:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00]

In [31]:
# softmax演示， scale的重要性
score = torch.randn(5)
print(score)
alpha1 = 0.1
alpha2 = 10
prob1 = F.softmax(score*alpha1,-1)
prob2 = F.softmax(score*alpha2,-1)
print(prob1,prob2)# 可以看到softmax函数输入值越大，其方差越大，这也是为什么attention中要进行根号dk缩放。类似雅可比矩阵也会都变的很小，导致梯度消失


tensor([-1.4421, -1.1066, -1.4678,  0.0533,  0.8438])
tensor([0.1835, 0.1898, 0.1830, 0.2131, 0.2306]) tensor([1.1807e-10, 3.3821e-09, 9.1314e-11, 3.6867e-04, 9.9963e-01])


In [57]:
# 构造self attention 的 mask
# mask的shape： 【batch_size, max_src_len, max_src_len】,值为1或-inf
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max_src_seq_len-L)),0) for L in src_len]),2)
print(valid_encoder_pos.shape)# 有效位置
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos,valid_encoder_pos.transpose(1,2))#两个有效矩阵相乘，计算出两两相关性
print(valid_encoder_pos_matrix)
invalid_encoder_pos = 1 - valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos.to(torch.bool)
print(mask_encoder_self_attention)

# 举个例子
score = torch.randn(batch_size, max_src_seq_len,max_src_seq_len)
masked_score = score.masked_fill(mask_encoder_self_attention,-1e9)
prob = F.softmax(masked_score,-1)
print(score)
print(masked_score)
print(prob)#可以看到pad的地方都为0，被mask调了。第三个和第四个单词均衡的原因是因为都被mask掉了，计算出的是均匀权重

torch.Size([2, 5, 1])
tensor([[[1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0.]]])
tensor([[[False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True],
         [ True,  True,  True,  True,  True]]])
tensor([[[-0.6656,  0.1447,  2.5085,  1.0907, -1.7251],
         [-1.0825,  1.4406, -0.6529, -0.6054,  0.1158],
         [-1.3831, -0.0417, -0.6725, -0.8332,  0.2140],
         [-0.1505,  3.3522, -0.1

In [62]:
# 构造intra attention 的mask
# Q @ K^T shape： 【batch_size, tgt_seq_len, src_seq_len】
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max_src_seq_len-L)),0) for L in src_len]),2)
valid_dencoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0,max_tgt_seq_len-L)),0) for L in tgt_len]),2)
print(valid_dencoder_pos.shape)
valid_cross_pos = torch.bmm(valid_dencoder_pos,valid_encoder_pos.transpose(1,2))
print(valid_cross_pos)
invalid_cross_pos = 1- valid_cross_pos
mask_cross_attention = invalid_cross_pos.to(torch.bool)
print(mask_cross_attention)

torch.Size([2, 5, 1])
tensor([[[1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])
tensor([[[False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]])


In [73]:
# 构造decoder self attention的mask
valid_decoder_tri_matrix = torch.cat([torch.unsqueeze(F.pad(torch.tril(torch.ones((L,L))),\
    (0,max_tgt_seq_len-L,0,max_tgt_seq_len-L)),0) for L in tgt_len])
print(valid_decoder_tri_matrix.shape)
print(valid_decoder_tri_matrix)
invalid_decoder_tri_matrix = 1- valid_decoder_tri_matrix
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)
print(invalid_decoder_tri_matrix)
# 举个例子
score = torch.randn(batch_size, max_tgt_seq_len,max_tgt_seq_len)
masked_score = score.masked_fill(invalid_decoder_tri_matrix,-1e9)
prob = F.softmax(masked_score,-1)
print(prob)

torch.Size([2, 5, 5])
tensor([[[1., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])
tensor([[[False,  True,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False, False,  True],
         [ True,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]])
tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9115, 0.0885, 0.0000, 0.0000, 0.0000],
         [0.2252, 0.4641, 0.3107, 0.0000, 0.0000],
         [0.0669, 0.0874, 0.6038, 0.2419, 0.000

In [ ]:
# 构建scaled self attention
def scaled_dot_product_attention(Q,K,V,attn_mask):
    # shape of Q,K,V : (batch_size*num_head, seq_len, model_dim/num_head)
    score = torch.bmm(Q,K.traspose(-2,-1))/torch.sqrt(model_dim)
    masked_score = score.masked_fill(attn_mask,-1)
    prob = F.softmax(masked_score,-1)
    context = torch.bmm(prob,V)
    return context
